In [269]:
# Importar librerías necesarias
import dash
import pandas as pd
import re
import os
from os import path, getcwd
import random
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objects as go
import plotly.graph_objs as go



try:
    import psycopg2
except:
    os.system("pip3 install psycopg2")
    import psycopg2


In [270]:
directorio_padre = path.dirname(getcwd())
directorio_padre

'/Users/esteban/Personal/Octavo Semestre/Analitica/Proyecto 2/actd_proyecto_2'

In [271]:
os.listdir()


['a', 'dashboard_revisado.ipynb', 'Predicciones.py']

In [272]:
import sys
sys.path.append(directorio_padre+'/modelamiento')

from  funciones import *
from  mod_explicativo import *
from  mod_predictivo import *

In [273]:
# Cargar variables de entorno
#load_dotenv()

# Conexión a la base de datos
connection = psycopg2.connect(
    dbname="datos_limpios",
    user="postgres",
    password="1234567890",
    host="proyecto2.cpgmg4am8kak.us-east-1.rds.amazonaws.com",
    port="5432"
)


In [274]:
# Consultar la base de datos para obtener los detalles del cliente
query = f"SELECT sex, education, marriage, age, limit_bal FROM datos_limpios"
customer_details = pd.read_sql_query(query, connection)
customer_details

/var/folders/rk/1txw87pj7sgdsrd0ny5897580000gn/T/ipykernel_8431/4197408056.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,sex,education,marriage,age,limit_bal
0,2,2,1,24,20000
1,2,2,2,26,120000
2,2,2,2,34,90000
3,2,2,1,37,50000
4,1,2,1,57,50000
...,...,...,...,...,...
29995,1,3,1,39,220000
29996,1,3,2,43,150000
29997,1,2,2,37,30000
29998,1,3,1,41,80000


In [275]:
import json
import os
import numpy as np

dict_importancia=result_mod_descr()

def write_dict_to_json(data_dict, file_path):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Prepare a JSON serializable dictionary by converting numpy floats to regular floats
    serializable_dict = {key: float(value) if isinstance(value, np.floating) else value for key, value in data_dict.items()}
    
    # Write the dictionary to a JSON file
    with open(file_path, 'w') as file:
        json.dump(serializable_dict, file, indent=4)

write_dict_to_json(dict_importancia, directorio_padre + '/data/feature_importance.json')


In [276]:
import json
def read_dict_from_json(file_path):
    # Read the dictionary from a JSON file
    with open(file_path, 'r') as file:
        data_dict = json.load(file)
    return data_dict

# Example usage:
imported_dict = read_dict_from_json('/Users/esteban/Downloads/feature_importance.json')
imported_dict

{'ID': 0.07086113095283508,
 'LIMIT_BAL': 0.06572240591049194,
 'SEX': 0.06495999544858932,
 'EDUCATION': 0.07412325590848923,
 'MARRIAGE': 0.09221955388784409,
 'AGE': 0.07129248976707458,
 'PAY_0': 0.0664275661110878,
 'PAY_2': 0.07302878051996231,
 'PAY_3': 0.06327730417251587,
 'PAY_4': 0.0693788006901741,
 'PAY_5': 0.07714333385229111,
 'PAY_6': 0.06943554431200027,
 'BILL_AMT1': 0.06437050551176071,
 'BILL_AMT2': 0.08987466990947723,
 'BILL_AMT3': 0.06752868741750717,
 'BILL_AMT4': 0.06951820105314255,
 'BILL_AMT5': 0.0801435261964798,
 'BILL_AMT6': 0.07113902270793915,
 'PAY_AMT1': 0.07887425273656845,
 'PAY_AMT2': 0.07130945473909378,
 'PAY_AMT3': 0.06621762365102768,
 'PAY_AMT4': 0.07807572185993195,
 'PAY_AMT5': 0.0694374218583107,
 'PAY_AMT6': 0.0688885748386383}

In [277]:


import plotly.graph_objects as go

def plot_feature_importance(importances, highlight=['EDUCATION', 'AGE']):
    # Convert the dictionary to a list of tuples and sort it
    sorted_importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)

    # Extract features and their corresponding importances
    features = [feat for feat, imp in sorted_importances]
    values = [imp for feat, imp in sorted_importances]

    # Set colors and opacity based on highlight
    colors = ['rgba(0, 0, 255, 0.4)' if feature not in highlight else 'rgba(0, 0, 255, 1.0)' for feature in features]
    borders = ['white'] * len(features)  # White borders for all bars

    # Create the bar plot
    fig = go.Figure(go.Bar(
        x=values,
        y=features,
        orientation='h',
        marker=dict(color=colors, line=dict(color=borders, width=1)),
        opacity=0.8  # General opacity setting
    ))

    # Update layout
    fig.update_layout(
        title='Feature Importance',
        xaxis_title='Importance',
        yaxis_title='Features',
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white'
    )

    return fig

# Plotting the feature importance
feature_importance_plot = plot_feature_importance(imported_dict)


In [278]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

sidebar = html.Div(
    [
        html.H1("Analítica Riesgo"),
        dbc.Nav(
            [
                dbc.NavLink("Información General", href="/info", active="exact"),
                dbc.NavLink("Explicativo", href="/explic", active="exact"),
                dbc.NavLink("Predictivo", href="/predict", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style={"width": "20%", "position": "fixed", "height": "100%", "background-color": "#f8f9fa"}
)

content = html.Div(id="page-content", style={"margin-left": "20%", "width": "80%"})

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    sidebar,
    content,
])

@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    if pathname == "/info":
        return html.Div([
            html.H1("Información General"),
            html.H2("Pregunta Descriptiva:"),
            html.P("¿Existe una importancia significativa entre la edad, nivel educativo de los clientes y su propensión al default en los últimos meses?"),
            html.H2("Pregunta Predictiva:"),
            html.P("¿Cuál es la probabilidad de default en clientes de tarjetas de crédito, considerando variables clave como el historial de pagos, el estado de la cuenta bancaria, la cantidad de pagos anteriores y el monto de crédito otorgado, mientras se considera la relación temporal de los pagos y la edad del cliente?"),
        ])
    elif pathname == "/explic":
        return html.Div([
            
            html.Br(),
            html.Hr(),
            html.H1("Análisis Explicativo"),

            html.Br(),
            html.Hr(),
            html.H4("Indice de Importancia por Categorias"),
            
            # Insert the Plotly graph here
            html.Div([
                dcc.Graph(
                    id='feature-importance-graph',
                    figure=feature_importance_plot  # This assumes your Plotly figure is named 'feature_importance_plot'
                )
            ], id='feature_importance'),

            html.Br(),
            html.Hr(),
            html.H4("Subtítulo 2"),
            html.P("Content for Subtítulo 2")
        ])
    elif pathname == "/predict":
        return html.Div([

            html.Br(),
            html.Hr(),
            html.H4("Probabilidad de Incumplimiento de Pago"),
            html.Label("Seleccionar ID del Cliente"),
            dcc.Input(id='customer-id', type='number', value=12),

            html.Br(),
            html.Hr(),
            html.Div(id='default-probability'),
            html.H4("Detalles del Cliente"),
            html.Table(id='customer-details'),
            
            html.Br(),
            html.Hr(),
            html.H4("Monto de Crédito Otorgado"),
            html.Table(id='credit-limit'),

            html.Br(),
            html.Hr(),
            html.H4("Estado de Cuenta y Montos Pagados a lo largo del Tiempo"),
            dcc.Graph(id='line-chart')

            
        ])
    else:
        return "404 Page Not Found"


In [279]:
import plotly.graph_objects as go  # Ensure this import is at the beginning of your file

# Callbacks para actualizar los componentes del dashboard
@app.callback(
    Output('default-probability', 'children'),
    Output('customer-details', 'children'),
    Output('line-chart', 'figure'),
    Output('credit-limit', 'children'),
    Input('customer-id', 'value')
)
def update_customer_info(customer_id):


    # Consultar la base de datos para obtener los detalles del cliente
    query = f"SELECT sex, education, marriage, age, limit_bal FROM datos_limpios WHERE id = {customer_id}"
    customer_details = pd.read_sql_query(query, connection)
    
    # Mapear los valores numéricos a sus respectivas categorías
    sex_mapping = {1: 'Masculino', 2: 'Femenino'}
    education_mapping = {1: 'Escuela de Graduados', 2: 'Universidad', 3: 'Escuela Secundaria', 4: 'Otros'}
    marriage_mapping = {1: 'Casado', 2: 'Soltero', 3: 'Otros'}

    # Aplicar el mapeo a las columnas correspondientes
    customer_details['sex'] = customer_details['sex'].map(sex_mapping)
    customer_details['education'] = customer_details['education'].map(education_mapping)
    customer_details['marriage'] = customer_details['marriage'].map(marriage_mapping)

    # Modificar el nombre de las variables en el DataFrame
    customer_details.rename(columns={'sex': 'Sexo', 'education': 'Nivel Educativo', 'marriage': 'Estado Civil','age': 'Edad', 'limit_bal': 'Monto Crédito'}, inplace=True)
    
    # Example database query and processing logic
    # Assuming the function proba_dado_id() returns a probability as a float
    default_probability = round(proba_dado_id(customer_id) * 100, 3)

    # Create gauge plot for the default probability
    gauge_figure = go.Figure(go.Indicator(
        mode="gauge+number",
        value=default_probability,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Default Probability"},
        gauge={'axis': {'range': [None, 100]},
               'bar': {'color': "darkblue"},
               'steps': [
                   {'range': [0, 50], 'color': 'lightgreen'},
                   {'range': [50, 75], 'color': 'yellow'},
                   {'range': [75, 100], 'color': 'red'},
               ],
               'threshold': {
                   'line': {'color': "black", 'width': 4},
                   'thickness': 0.75,
                   'value': default_probability}
        }
    ))

    gauge_plot = dcc.Graph(figure=gauge_figure)

    # Actualizar el número con la probabilidad de incumplimiento de pago
    default_prob_component = [html.H2(f"Probabilidad de Incumplimiento de Pago: {default_probability}%"), gauge_plot ]

    # Actualizar componente con detalles del cliente
    customer_details_component = dbc.Table.from_dataframe(
        customer_details[['Sexo', 'Nivel Educativo',  'Estado Civil', 'Edad']],
        striped=True,
        bordered=True,
        hover=True,
        responsive=True,
        style={'width': '100%', 'margin': 'auto'}
    )
    
         # Actualizar tabla con el monto de crédito otorgado
    credit_limit_component = html.Div([
        dbc.Table.from_dataframe(
            pd.DataFrame({'Monto de Crédito Otorgado': [customer_details['Monto Crédito'][0]]}),
            striped=True,
            bordered=True,
            hover=True,
            responsive=True,
            style={'width': '100%', 'margin': 'auto'}
        )
    ])

    # Consultar la base de datos para obtener los datos de bill_amt y pay_amt
    query_bill_amt = f"SELECT bill_amt1, bill_amt2, bill_amt3, bill_amt4, bill_amt5, bill_amt6 FROM datos_limpios WHERE id = {customer_id}"
    query_pay_amt = f"SELECT pay_amt1, pay_amt2, pay_amt3, pay_amt4, pay_amt5, pay_amt6 FROM datos_limpios WHERE id = {customer_id}"
    bill_amt_data = pd.read_sql_query(query_bill_amt, connection)
    pay_amt_data = pd.read_sql_query(query_pay_amt, connection)

    # Crear el DataFrame con las columnas solicitadas
    df = {
        'Meses': ['Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre'],
        'Bill Amount': bill_amt_data.values.flatten(),
        'Pay Amount': pay_amt_data.values.flatten()
    }

    # Crear el gráfico de línea para Bill_amt y Pay_amt
    line_plot_figure = go.Figure()
    line_plot_figure.add_trace(go.Scatter(x=df['Meses'], y=df['Bill Amount'], mode='lines', name='Bill Amount', line=dict(width=2)))
    line_plot_figure.add_trace(go.Scatter(x=df['Meses'], y=df['Pay Amount'], mode='lines', name='Pay Amount', line=dict(width=2)))

    # Actualizar el diseño del gráfico
    line_plot_figure.update_layout(
        margin=dict(t=30, l=30, r=30, b=30),  # Margins
        legend=dict(x=0.75, y=1, bgcolor='rgba(255, 255, 255, 0.5)'),  # Legend
        xaxis_title="Meses",
        yaxis_title="Valor"
    )

    # Ajustar el tamaño de los puntos
    line_plot_figure.update_traces(marker=dict(size=12, opacity=0.9))



    return default_prob_component, customer_details_component,  line_plot_figure, credit_limit_component


In [280]:
import socket
import requests

def get_local_ip():
    try:
        # Create a socket connection.
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
            # Connect to a Google DNS server to determine the local IP.
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
    except Exception as e:
        return "Unable to determine local IP"

local_ip = get_local_ip()

In [ ]:

print(f"Local IP (for access within local network): http://{local_ip}:8051")
# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=8051, debug=True)

Local IP (for access within local network): http://192.168.1.48:8051


/var/folders/rk/1txw87pj7sgdsrd0ny5897580000gn/T/ipykernel_8431/668373813.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



1/1 [==============================] - 0s 73ms/step
Probabilidad de default para el cliente con ID 12: 16.65%


/var/folders/rk/1txw87pj7sgdsrd0ny5897580000gn/T/ipykernel_8431/668373813.py:85: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/var/folders/rk/1txw87pj7sgdsrd0ny5897580000gn/T/ipykernel_8431/668373813.py:86: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

